In [1]:
### Imports
import streamlit as st
import os
import io
import json
import pandas as pd
from datetime import datetime
import chromadb

import langchain
# from langchain.cache import InMemoryCache
# langchain.llm_cache = InMemoryCache()
# from langchain.cache import SQLiteCache
# langchain.llm_cache = SQLiteCache(database_path="langchain.db")
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, Tool, tool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.llms import OpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain import LLMMathChain
from langchain.vectorstores import Chroma
from langchain.agents.agent_toolkits import create_python_agent
from contextlib import redirect_stdout
from typing import Optional, Type
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

## Initialize

In [18]:
### Read runbook and create vector index
raw_documents = TextLoader('content/runbook.txt').load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
runbook_vectors = Chroma.from_documents(documents, OpenAIEmbeddings())


In [3]:
# OpenAI Credentials
if not os.environ["OPENAI_API_KEY"]:
    openai_api_key = st.secrets["OPENAI_API_KEY"]
else:
    openai_api_key = os.environ["OPENAI_API_KEY"]


## Planner

In [4]:
def planner(myquestion):

    ### Bring in my controlling documents and the additonal template
    relevancy_cutoff = .8
    
    docs = runbook_vectors.similarity_search_with_relevance_scores(myquestion, k=3)

    mytasks = ""
    for x,v in docs:
        if mytasks == "":
            mytasks = str(x) + "\n\n"
            continue
        if v > relevancy_cutoff:
            mytasks = mytasks + str(x) + "\n\n"
            
    template="""
    You are a helpful chatbot that collects information from the user based upon the type of report 
    they need to comnplete. Use the TEXT below to answer the user's PROMPT.
    If you can answer their question, then simply provide the answer as a paragraph. 
    If you need to create a series of steps necessary for answering the question, 
    return a list of steps with each step labeled in the format "Step #:".
    If the user's request cannot be addressed by the TEXT, then only respond with 
    "Sorry, but I can't help you with that task."

    TEXT:
    {mytasks}

    PROMPT:
    {myquestion}
    """
        
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.9, verbose=True)

    chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(template))
    
    response = chain(inputs={"mytasks": mytasks,"myquestion": myquestion})

    return response



In [5]:
# prompt="What kind of reports can you support?"
prompt="I would like to create a burglary report."
# prompt="How do I create a crime scene report?"
# prompt="I would like to create a financial fraud report."

In [6]:
response = planner(prompt)
# print(response['text'])
steps = response['text'].split("\n")
df = pd.DataFrame(steps, columns=['steps'])
df.replace('(^\s+|\t+|\s+$)', '', regex=True, inplace=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 1000)
print("PROMPT:",prompt+"\n")
print("STEPS:")
for index, row in df.iterrows():
    print(df['steps'].values[index])


PROMPT: I would like to create a burglary report.

STEPS:
Step 1: Ask the user for the location of the burglary, including the street address, city, and state. If the user cannot provide a street address, ask for a nearby landmark or cross streets.
Step 2: Ask the user for the date and time of the burglary, even if it is just an estimate.
Step 3: Ask the user to list the property stolen. Have the user provide a description of each item, when it was purchased, and an estimated value for each item.
Step 4: Calculate the total value of all stolen items.
Step 5: Ask the user for their contact information, including their name, address, and phone number.
Step 6: Ask the user to list the contact information for any witnesses, including their name, address, phone number, and relationship to the user.


## Executor

In [7]:
def get_input(myprompt) -> str:
    print(myprompt)
    contents = input()
    return contents


# x=get_input("Enter list of items:")

def evaluator(myquestion, myanswer):

    template="""
    Evaluate the user's RESPONSE below with the QUESTION the user was asked. 
    If the answer adequately answers the question OR the user indicates they cannot answer the question, then respond that the answer is adequate.
    If the RESPONSE DOES NOT adequately answer the QUESTION then respond with what information is missing from the answers.
   

    QUESTION:
    {myquestion}

    RESPONSE:
    {myanswer}
    """
    model="gpt-4"
    llm = ChatOpenAI(model=model, temperature=0.0, verbose=False)
    ev_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(template))
    response = ev_chain(inputs={"myanswer": myanswer,"myquestion": myquestion})

    return response



In [59]:
def final_report(mydf):
    mychat = ""
    for index, row in mydf.iterrows():
        mychat = mychat+"Interviewer: "+row['steps']+"\n"
        mychat = mychat+"Human: "+row['response']+"\n"
    
    template="""
    Using ALL the information under CONVERSATION, write am incident report that 
    logically organizes the information from the conversation
   

    CONVERSATION:
    {mychat}

    """
    model="gpt-4"
    llm = ChatOpenAI(model=model, temperature=0.0, verbose=False)
    ev_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(template))
    response = ev_chain(inputs={"mychat": mychat})

    return response



## Lambda style

In [53]:
def new_lambda(myquestion, myanswer, myhistory="", myindex=0):

    if len(myhistory)== 0:
        myresults = planner(myanswer)
        # print(myresults)
        steps = myresults['text'].split("\n")
        # print(steps)
        mydf = pd.DataFrame(steps, columns=['steps'])
        mydf['response']=''
        mydf.replace('(^\s+|\t+|\s+$)', '', regex=True, inplace=True)
        if len(mydf.index) == 1:
            return myresults['text'], 'True', '', 0
        elif len(mydf.index) == 0:
            return "I'm sorry but I can't help you. Use this tool to assist in collecting incident reports.", 'True', '', 0
        else:
            result = mydf.to_json(orient="records")
            return mydf['steps'].values[0], 'False', result, 0
    else:
        mydf = pd.DataFrame.from_dict(json.loads(history), orient='columns')
        mydf['response'][myindex]=mydf['response'][myindex]+" "+myanswer
        evaluation = evaluator(myquestion, mydf['response'][myindex])
        myresults = evaluation['text']
        if "adequate" in myresults:
            myindex = myindex+1
            if myindex < len(mydf.index):
                result = mydf.to_json(orient="records")
                return mydf['steps'].values[myindex], False, result, myindex
            else:
                report = final_report(mydf)
                return report['text'], True, "", 0
        else:
            result = mydf.to_json(orient="records")
            return myresults, False, result, myindex
        

In [11]:
myquestion = "how can I help you?"
myanswer = "What kind of reports can you support?"

answer, final, history, qnbr = new_lambda(myquestion, myanswer)
print(answer)

print("\nFinal:",final,"Question #:",qnbr)
print(history)

The system is currently configured to collect information on burglaries and crime scenes. However, the system can be expanded to include any type of report as long as instructions are provided in the runbook.

Final: True Question #: 0



In [44]:
myquestion = "how can I help you?"
myanswer = "Help me complete a burglary report"

answer, final, history, qnbr = new_lambda(myquestion, myanswer)
print(answer)

print("\nFinal:",final,"Question #:",qnbr)
print(json.dumps(json.loads(history),indent=2))

Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.

Final: False Question #: 0
[
  {
    "steps": "Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.",
    "response": ""
  },
  {
    "steps": "Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.",
    "response": ""
  },
  {
    "steps": "Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.",
    "response": ""
  },
  {
    "steps": "Step 4: Ask the user for their contact information, including name, address, and phone number.",
    "response": ""
  },
  {
    "s

In [45]:
myquestion = answer
myanswer = "123 Main St."
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)

print("\nFinal:",final,"Question #:",qnbr)
print(json.dumps(json.loads(history),indent=2))

The city and state are missing from the user's response.

Final: False Question #: 0
[
  {
    "steps": "Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.",
    "response": " 123 Main St."
  },
  {
    "steps": "Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.",
    "response": ""
  },
  {
    "steps": "Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.",
    "response": ""
  },
  {
    "steps": "Step 4: Ask the user for their contact information, including name, address, and phone number.",
    "response": ""
  },
  {
    "steps": "Step 5: Ask the user to list the contact information for any witnesses, including name, address, phone numbers, and relationship to the 

In [46]:
myquestion = answer
myanswer = "Buffalo, NY"
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)

print("\nFinal:",final,"Question #:",qnbr)
print(json.dumps(json.loads(history),indent=2))

Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.

Final: False Question #: 1
[
  {
    "steps": "Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.",
    "response": " 123 Main St. Buffalo, NY"
  },
  {
    "steps": "Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.",
    "response": ""
  },
  {
    "steps": "Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.",
    "response": ""
  },
  {
    "steps": "Step 4: Ask the user for their contact information, including name, address, and phone number.",
    "response": ""
  },
  {
    "steps": "Step 5: Ask the user to list the contact information for any witnesses, including n

In [47]:
myquestion = answer
myanswer = "01/01/2023"
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)

print("\nFinal:",final,"Question #:",qnbr)
print(json.dumps(json.loads(history),indent=2))

The time of the burglary is missing from the answer.

Final: False Question #: 1
[
  {
    "steps": "Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.",
    "response": " 123 Main St. Buffalo, NY"
  },
  {
    "steps": "Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.",
    "response": " 01/01/2023"
  },
  {
    "steps": "Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.",
    "response": ""
  },
  {
    "steps": "Step 4: Ask the user for their contact information, including name, address, and phone number.",
    "response": ""
  },
  {
    "steps": "Step 5: Ask the user to list the contact information for any witnesses, including name, address, phone numbers, and r

In [48]:
myquestion = answer
myanswer = "I don't know the time of the burglary"
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)

print("\nFinal:",final,"Question #:",qnbr)
print(json.dumps(json.loads(history),indent=2))

Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.

Final: False Question #: 2
[
  {
    "steps": "Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.",
    "response": " 123 Main St. Buffalo, NY"
  },
  {
    "steps": "Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.",
    "response": " 01/01/2023 I don't know the time of the burglary"
  },
  {
    "steps": "Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.",
    "response": ""
  },
  {
    "steps": "Step 4: Ask the user for their contact information, including name, address, and phone number.",
    "respon

In [49]:
myquestion = answer
myanswer = "1 gold watch with a value of $400. I don't know when it was purchased. It was a gift on Christmas, 2005."
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)
print(final)
print(history)
print(qnbr)

Step 4: Ask the user for their contact information, including name, address, and phone number.
False
[{"steps":"Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.","response":" 123 Main St. Buffalo, NY"},{"steps":"Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.","response":" 01\/01\/2023 I don't know the time of the burglary"},{"steps":"Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.","response":" 1 gold watch with a value of $400. I don't know when it was purchased. It was a gift on Christmas, 2005."},{"steps":"Step 4: Ask the user for their contact information, including name, address, and phone number.","response":""},{"steps":"Step 5: Ask the user to list the 

In [50]:
myquestion = answer
myanswer = "Rory McLean, 3046 Terra Maria Way, Ellicott City, MD. (443) 280-0781"
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)
print(final)
print(history)
print(qnbr)

Step 5: Ask the user to list the contact information for any witnesses, including name, address, phone numbers, and relationship to the user.
False
[{"steps":"Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.","response":" 123 Main St. Buffalo, NY"},{"steps":"Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.","response":" 01\/01\/2023 I don't know the time of the burglary"},{"steps":"Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.","response":" 1 gold watch with a value of $400. I don't know when it was purchased. It was a gift on Christmas, 2005."},{"steps":"Step 4: Ask the user for their contact information, including name, address, and phone number.","response":

In [61]:
myquestion = answer
myanswer = "There were no witnesses to the crime."
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)
print(final)
print(history)
print(qnbr)

INCIDENT REPORT:

Date of Report: Current Date

Incident Type: Burglary

Location of Incident: 123 Main St. Buffalo, NY

Date and Time of Incident: 01/01/2023, Time Unknown

Details of Incident: The victim reported a burglary at the above-mentioned address. The exact time of the incident is unknown. The stolen property is a gold watch, valued at approximately $400. The watch was received as a gift in Christmas, 2005, and the exact purchase date is unknown.

Victim Information: 
Name: Rory McLean
Address: 3046 Terra Maria Way, Ellicott City, MD
Phone Number: (443) 280-0781

Witness Information: According to the victim, there were no witnesses to the crime.

This report is based on the information provided by the victim during the interview. Further investigation is required to identify the perpetrator and recover the stolen property.
True

0


In [43]:
myquestion = "how can I help you?"
myanswer = "What is your definition of burglary?"

answer, final, history, qnbr = new_lambda(myquestion, myanswer)
print(answer)

print("\nFinal:",final,"Question #:",qnbr)
print(history)

A burglary is defined as the crime of entering a structure, such as a house or commercial building, with the intent to commit a felony, such as theft.

Final: True Question #: 0

